Load packages

In [60]:
import sklearn
import pandas as pd
import numpy as np
from math import pi

import matplotlib.pyplot as plt
import seaborn as sns

In [62]:
columns = ['GAME_DATE_EST',
 'GAME_ID',
 'HOME_TEAM_WINS',
 'HOME_TEAM',
 'HOME_TEAM_WIN_LAST_10_GAMES',
 'VISITOR_TEAM_WINS',
 'VISITOR_TEAM',
 'VISITOR_TEAM_WIN_LAST_10_GAMES']

ranking = pd.read_csv('../nba-games/ranking.csv')
teams = pd.read_csv('../nba-games/teams.csv', usecols=['TEAM_ID', 'ABBREVIATION'])
teams.rename(columns={"ABBREVIATION":"NAME"}, inplace=True)
games = pd.read_csv("../nba-games/games.csv",parse_dates=["GAME_DATE_EST"]
                    ,infer_datetime_format=True)
games = games.drop_duplicates()\
    .drop(columns=["TEAM_ID_home", "TEAM_ID_away", "GAME_STATUS_TEXT"], axis=1)

df = games[games.SEASON == 2018]
df.loc[:, ("VISITOR_TEAM_WINS")] = (df["HOME_TEAM_WINS"]==0).astype(int)
print(teams.shape)
print(games.shape)
print(df.shape)

(30, 2)
(23195, 18)
(1378, 19)


In [ ]:
def calculate_mean_last_10_games(s, prefix, suffix):
    query = (game_date > df.GAME_DATE_EST) & \
            ((df.HOME_TEAM == team) | \
            (df.VISITOR_TEAM == team))
    team_df = df.loc[query]
    win_lost_balance = 0
    pts_balance = 0
    reb_balance = 0
    fg_balance = 0
    fg3_balance = 0
    ast_balance = 0
    for idx, row in team_df.head(10).iterrows():
        if row.HOME_TEAM == team:
            win_lost_balance += row.HOME_TEAM_WINS
            pts_balance += row["PTS_" + suffix]
            pts_balance += row["REB_" + suffix]
            pts_balance += row["FG_PCT_" + suffix]
            pts_balance += row["FG3_PCT_" + suffix]
            pts_balance += row["AST_" + suffix]
        else:
            win_lost_balance += row.VISITOR_TEAM_WINS 
            pts_balance += row["PTS_" + suffix]
            pts_balance += row["REB_" + suffix]
            pts_balance += row["FG_PCT_" + suffix]
            pts_balance += row["FG3_PCT_" + suffix]
            pts_balance += row["AST_" + suffix]
    s[prefix + "TEAM_WIN_LAST_10_GAMES"] = win_lost_balance
    s[prefix + "TEAM_WIN_LAST_10_GAMES"] = win_lost_balance
    
    return s

In [65]:
def calculate_mean_last_10_games(s):
    query = (game_date > df.GAME_DATE_EST) & \
            ((df.HOME_TEAM == team) | \
            (df.VISITOR_TEAM == team))
    team_df = df.loc[query]
    win_lost_balance = 0
    for idx, row in team_df.head(10).iterrows():
        if row.HOME_TEAM == team:
            win_lost_balance += row.HOME_TEAM_WINS
        else:
            win_lost_balance += row.VISITOR_TEAM_WINS 
    return s

df.apply(calculate_mean_last_10_games, axis=1)

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,VISITOR_TEAM_WINS
16196,2019-06-13,41800406,1610612744,1610612761,2018,110.0,0.488,0.700,0.355,28.0,42.0,114.0,0.476,0.793,0.394,25.0,39.0,0,1
16197,2019-06-10,41800405,1610612761,1610612744,2018,105.0,0.447,0.778,0.250,19.0,43.0,106.0,0.463,0.714,0.476,27.0,37.0,0,1
16198,2019-06-07,41800404,1610612744,1610612761,2018,92.0,0.449,0.667,0.296,26.0,42.0,105.0,0.419,0.958,0.313,22.0,39.0,0,1
16199,2019-06-05,41800403,1610612744,1610612761,2018,109.0,0.396,0.833,0.333,25.0,41.0,123.0,0.524,0.952,0.447,30.0,40.0,0,1
16200,2019-06-02,41800402,1610612761,1610612744,2018,104.0,0.372,0.885,0.289,17.0,49.0,109.0,0.463,0.870,0.382,34.0,42.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17569,2018-09-30,11800009,1610612741,1610612740,2018,128.0,0.495,0.963,0.222,23.0,52.0,116.0,0.454,0.789,0.333,24.0,45.0,1,0
17570,2018-09-30,11800010,1610612747,1610612743,2018,107.0,0.460,0.731,0.320,26.0,37.0,124.0,0.460,0.886,0.351,25.0,50.0,0,1
17571,2018-09-29,11800003,1610612761,1610612757,2018,122.0,0.415,0.824,0.343,15.0,45.0,104.0,0.420,0.720,0.387,25.0,46.0,1,0
17572,2018-09-29,11800005,1610612744,1610612750,2018,110.0,0.473,0.769,0.353,33.0,48.0,114.0,0.426,0.733,0.400,15.0,47.0,0,1


In [52]:
df = pd.merge(df, teams, left_on='HOME_TEAM_ID', right_on='TEAM_ID', how='inner')[
    ['GAME_DATE_EST', 'GAME_ID', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'SEASON',
       'PTS_home', 'FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home', 'AST_home',
       'REB_home', 'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away',
       'AST_away', 'REB_away', 'HOME_TEAM_WINS', 'VISITOR_TEAM_WINS']]

In [53]:
def add_team_abbr_column(is_home):
    global df
    if is_home:
        df = pd.merge(df, teams, left_on='HOME_TEAM_ID', right_on='TEAM_ID')
        df.rename(columns={"NAME":"HOME_TEAM"}, inplace=True)
        df = df.drop(columns=["HOME_TEAM_ID"], axis=1)
    else:
        df = pd.merge(df, teams, left_on='VISITOR_TEAM_ID', right_on='TEAM_ID')
        df.rename(columns={"NAME":"VISITOR_TEAM"}, inplace=True)
        df = df.drop(columns=["VISITOR_TEAM_ID"], axis=1)
    df = df.drop(columns=["TEAM_ID"], axis=1)


add_team_abbr_column(True)
add_team_abbr_column( False)
df = df.sort_values("GAME_DATE_EST", ascending=False).reset_index(drop=True)

In [54]:
df_preprocessed = pd.DataFrame()

df_preprocessed = df[["HOME_TEAM", "VISITOR_TEAM"]].copy()

df_preprocessed.sample()

,HOME_TEAM,VISITOR_TEAM
667,BKN,BOS


In [55]:
def last_10_games(team, game_date):
    query = (game_date > df.GAME_DATE_EST) & \
            ((df.HOME_TEAM == team) | \
            (df.VISITOR_TEAM == team))
    team_df = df.loc[query]
    balance = 0
    for idx, row in team_df.head(10).iterrows():
        if row.HOME_TEAM == team:
            balance += row.HOME_TEAM_WINS
        else:
            balance += row.VISITOR_TEAM_WINS
    return balance

In [56]:
#df_preprocessed["HOME_TEAM_WIN_LAST_10_GAMES"] = 0
#df_preprocessed["VISITOR_TEAM_WIN_LAST_10_GAMES"] = 0
for idx, row in df.iterrows():
   df.at[idx, "HOME_TEAM_WIN_LAST_10_GAMES"] = last_10_games(row["HOME_TEAM"], row["GAME_DATE_EST"])
   df.at[idx, "VISITOR_TEAM_WIN_LAST_10_GAMES"] = last_10_games(row["VISITOR_TEAM"], row["GAME_DATE_EST"])


In [57]:
test = df[columns]
query = ('2019-06-10' > df.GAME_DATE_EST) & \
            ((df.HOME_TEAM == "TOR") | \
            (df.VISITOR_TEAM == "TOR"))
test[query].head(20)

,GAME_DATE_EST,GAME_ID,HOME_TEAM_WINS,HOME_TEAM,HOME_TEAM_WIN_LAST_10_GAMES,VISITOR_TEAM_WINS,VISITOR_TEAM,VISITOR_TEAM_WIN_LAST_10_GAMES
2,2019-06-07,41800404,0,GSW,7.0,1,TOR,7.0
3,2019-06-05,41800403,0,GSW,7.0,1,TOR,6.0
4,2019-06-02,41800402,0,TOR,7.0,1,GSW,7.0
5,2019-05-30,41800401,1,TOR,7.0,0,GSW,8.0
6,2019-05-25,41800306,1,TOR,6.0,0,MIL,6.0
7,2019-05-23,41800305,0,MIL,7.0,1,TOR,5.0
8,2019-05-21,41800304,1,TOR,5.0,0,MIL,8.0
10,2019-05-19,41800303,1,TOR,5.0,0,MIL,9.0
12,2019-05-17,41800302,1,MIL,9.0,0,TOR,6.0
14,2019-05-15,41800301,1,MIL,8.0,0,TOR,7.0


In [58]:
#print(df_preprocessed.shape)
#print(df.shape)
#
#df[["VISITOR_TEAM_WIN_LAST_10_GAMES", "HOME_TEAM_WIN_LAST_10_GAMES"]]


df_preprocessed = pd.concat(
    [df_preprocessed,
     df[["HOME_TEAM_WIN_LAST_10_GAMES", "VISITOR_TEAM_WIN_LAST_10_GAMES"]]]
    , axis=1
)
df_preprocessed.head(5)





,HOME_TEAM,VISITOR_TEAM,HOME_TEAM_WIN_LAST_10_GAMES,VISITOR_TEAM_WIN_LAST_10_GAMES
0,GSW,TOR,7.0,7.0
1,TOR,GSW,7.0,7.0
2,GSW,TOR,7.0,7.0
3,GSW,TOR,7.0,6.0
4,TOR,GSW,7.0,7.0
